# Intro to AI — Assignment 1: Sliding Puzzle


## Set up

In [ ]:
try:
  import google.colab
  IN_COLAB= True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
import json
import copy
import numpy as np
import heapq as hq

## Part 1 - Reading and Validating Sliding-Puzzle Problems

In [ ]:
IS_VALID = True

In [ ]:
def puzzle_validity(board):
   key_vals = ['n', 'start', 'goal']

   # Validate fields
   for i in range(0, len(key_vals)):
      if board.get(key_vals[i]) == None: return False


   if board['n'] <= 1: return False


   # Check if puzzle matrix is square.
   # If index is out of range, matrix is not square.
   try:
      for i in range(0, len(board)):
         if len(board['start'][i]) % board['n'] != 0: return False
         if len(board['goal'][i]) % board['n'] != 0: return False
   except IndexError as e:
      print(f"{e}\nMatrix not square.")
      return False


   # Check if values in matrix are between 0 and n^2 - 1
   for i in range(0, len(board)):
      for j in range(0, len(board)):
         if board['start'][i][j] < 0 or board['start'][i][j] > (board['n']**2 - 1): return False
         if board['goal'][i][j] < 0 or board['goal'][i][j] > (board['n']**2 - 1): return False

   return True

In [ ]:
infile = open('/content/gdrive/MyDrive/Colab Notebooks/Assignment1/8-puzzle-files/25-moves.json')
board = json.load(infile)
IS_VALID = puzzle_validity(board)

## Part 2 - Sliding-Tile Puzzle Rules

In [ ]:
def get_blank_square(state):
  '''Return indices of empty square in given state.'''

  state = np.array(state)
  board = np.where(state==0)
  y, x = board[0][0], board[1][0]
  return y, x

In [ ]:
def actions(state):
  '''Given a state, locate empty tile and evaluate
  which direction(s) the tile can move.'''

  # state = np.array(state)
  # board = np.where(state==0)
  # y, x = board[0][0], board[1][0]

  y,x = get_blank_square(state)

  applicable_rules = []

  # Encoded rules
  if y != 0:
    applicable_rules.append('Up')
  if y != 2:
    applicable_rules.append('Down')
  if x != 0:
    applicable_rules.append('Left')
  if x != 2:
    applicable_rules.append('Right')

  return applicable_rules

In [ ]:
def successor_state(state, action):
  '''Given a state and applicable rule, apply rule and return successive state'''

  y, x = get_blank_square(state)
  successor = copy.deepcopy(state) # Copy current state

  # Apply action to successor state
  delta = {'Up': -1, 'Down': 1, 'Left': -1, 'Right': 1}

  if action == 'Up' or action == 'Down':
    y_swap, x_swap = (y + delta[action]), x
  if action == 'Left' or action == 'Right':
    y_swap, x_swap = y, (x + delta[action])

  successor[y][x], successor[y_swap][x_swap] = successor[y_swap][x_swap], successor[y][x]

  return successor

In [ ]:
def is_goal_state(state, goal_state):
  return state == goal_state

In [ ]:
def path_back(node):
      """Return a list of nodes forming the path from the root to this node."""
      path = []
      while node:
        if node.parent is not None:
          path.append(node.rule)
        node = node.parent
      return list(reversed(path))


In [ ]:
def search_path(node, state):
  while node.state != state:
    node = node.parent

  if node is not None:
    return node
  else:
    return None

## Part 3 - Backtracking Control Strategy

### Recursive Back Tracking

In [ ]:
def back_tracking(datalist, goal, depth_bound, count):
  count[0]+= 1

  data, parent = datalist[0]

  # Check if the current state has already been visited
  if data in [state for state, _ in datalist[1:]]:
    return None, count

  # Check if current state is goal state. Return empty list to append rules to.
  if is_goal_state(data, goal):
    print('------------------')
    print('Goal state reached')
    return [], count

  # If the current depth is greater than depth bound, step out of call.
  if len(datalist) > depth_bound:
    return None, count

  rules = actions(data) # Get applicable rules.

  # Loop as long as there are rules to apply.
  while rules:
    rule = rules.pop()

    rdata = successor_state(data, rule)

    if parent is not None and parent == rdata:
      continue

    # Append successive state to front of list.
    rdatalist = [(rdata, data)] + datalist[:]

    path, count = back_tracking(rdatalist, goal, depth_bound, count)
    # If goal state reached, append current rule to path.
    if path is not None:
      return path[:] + [rule], count

  return None, count

In [ ]:
if IS_VALID:
  start = [(board['start'], [])]
  path = []
  count = [0]

  print('Start:')
  print(np.matrix(board['start']))
  print('Goal:')
  print(np.matrix(board['goal']))

  path, count = back_tracking(start, board['goal'], 25, count)

  if path is not None:
    print(f"Solution length {len(path)}")
    print(f"Path: {path}")
    print(f"States examined: {count[0]}")
  else:
    print('No solution found at given depth bound')

else:
  print('Invalid puzzle data')


Start:
[[7 2 4]
 [0 5 6]
 [8 3 1]]
Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
------------------
Goal state reached
Solution length 25
Path: ['Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Down', 'Right', 'Up']
States examined: 3099004


### Iterative Back Tracking

In [ ]:
def it_back_tracking(datalist, goal, depth_bound):
  depth = 2
  count = [0]
  while depth <= depth_bound:

    path, count = back_tracking(datalist, goal, depth, count)

    if path is not None:
      print(f"Solution length: {len(path)}")
      print(f"Path: {path}")
      print(f"States examined: {count[0]}")
      print(f"Solution maximum depth: {depth}")
    else:
      print(f"No solution found with depth bound: {depth}")
    depth += 1

In [ ]:
if IS_VALID:
  start = [(board['start'], [])]
  print('Start:')
  print(np.matrix(board['start']))
  print('Goal:')
  print(np.matrix(board['goal']))
  it_back_tracking(start, board['goal'], 25)

else:
  print('Invalid puzzle data')

Start:
[[7 2 4]
 [0 5 6]
 [8 3 1]]
Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
No solution found with depth bound: 2
No solution found with depth bound: 3
No solution found with depth bound: 4
No solution found with depth bound: 5
No solution found with depth bound: 6
No solution found with depth bound: 7
No solution found with depth bound: 8
No solution found with depth bound: 9
No solution found with depth bound: 10
No solution found with depth bound: 11
No solution found with depth bound: 12
No solution found with depth bound: 13
No solution found with depth bound: 14
No solution found with depth bound: 15
No solution found with depth bound: 16
No solution found with depth bound: 17
No solution found with depth bound: 18
No solution found with depth bound: 19
No solution found with depth bound: 20
No solution found with depth bound: 21
No solution found with depth bound: 22
No solution found with depth bound: 23
No solution found with depth bound: 24
------------------
Goal state reached
Solu

## Part 4 - Graph Search

In [ ]:
class n_puzzle:

  def __init__(self, initial, goal=None):
    '''Define goal state and initialize problem'''
    self.initial = initial
    self.goal = goal

In [ ]:
class Node:

  def __init__(self, state, parent=None, rule=None, cost=0):
    self.state = state
    self.parent = parent
    self.rule = rule
    self.cost = cost
    if parent:
      self.cost = parent.cost + 1


In [ ]:
def graph_search(problem):

  # Initialize open with starting state.
  open = [(0, problem.initial, Node(problem.initial))]
  closed = set()
  count = 0

  # Loop as long as there are nodes in frontier.
  while open:

    count += 1

    # Pop first element
    cost, current_state, node = hq.heappop(open)
    closed.add(tuple(map(tuple, current_state)))

    # If goal state found, return path and number of states examined.
    if is_goal_state(current_state, problem.goal):
      return path_back(node), count

    rules = actions(current_state)

    # Create lists of board states.
    open_states = [state for _, state, obj in open]

    for rule in rules: # Expand frontier

      new_state = successor_state(current_state, rule)

      child = Node(new_state, node, rule)

      # If the resultant state is not in open or closed, add to open.
      if new_state not in open_states and tuple(map(tuple,new_state)) not in closed:
        hq.heappush(open, (child.cost, child.state, child))

      # If resultant state is in closed, evaluate costs of path and swap if
      # new path is shorter.
      if tuple(map(tuple,new_state)) in closed:
        visited_node = search_path(child, new_state)

        if child.cost < visited_node.cost:
          child.parent = node
          visited_node.parent = None

  return None, count

In [ ]:
if IS_VALID:

  puzzle = n_puzzle(board['start'], board['goal']) # Problem instance

  print('Start:')
  print(np.matrix(board['start']))
  print('Goal:')
  print(np.matrix(board['goal']))

  path, count = graph_search(puzzle)

  if path is not None:
    print(f"Solution length: {len(path)}")
    print(f"Path: {path}")
    print(f"States examined: {count}")
else:
  print('Invalid puzzle data')

Start:
[[7 1 0]
 [3 6 5]
 [8 2 4]]
Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
Solution length: 20
Path: ['Down', 'Left', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Up', 'Left', 'Down', 'Right', 'Up', 'Right', 'Up', 'Left', 'Left']
States examined: 37810


## Part 5 - Algorithm A*

### A* using h1

In [ ]:
def h1(node_state):
  count = 0
  flat_list = []

  for sublist in node_state:
    for item in sublist:
      flat_list.append(item)

  for i in range(1,len(flat_list)):
    if flat_list[i] != i:
      count += 1

  return count

# Source: https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists

In [ ]:
def h1_a_star_search(problem):

  # Initialize open with starting state
  open = [(0, problem.initial, Node(problem.initial))]
  closed = set()
  count = 0

  # Loop as long as there are nodes in frontier
  while open:

    count += 1

    # Pop first element
    cost, current_state, node = hq.heappop(open)
    closed.add(tuple(map(tuple, current_state)))

    # If goal state found, return path and number of states examined
    if is_goal_state(current_state, problem.goal):
      return path_back(node), count

    # Get possible moves
    rules = actions(current_state)

    # Create lists of board states
    open_states = [state for _, state, obj in open]

    for rule in rules: # Expand frontier

      new_state = successor_state(current_state, rule)

      child = Node(new_state, node, rule)

      # If the resultant state is not in open or closed, add to open
      if new_state not in open_states and tuple(map(tuple,new_state)) not in closed:
        heuristic_val = child.cost + h1(child.state)
        hq.heappush(open, (heuristic_val, child.state, child))

      # If the new state is in the closed set,
      # check if shorter path has been found
      if tuple(map(tuple,new_state)) in closed:
        visited_node = search_path(child, new_state)

        if visited_node is not None:
          heuristic_val = child.cost + h1(new_state)

          if heuristic_val < visited_node.cost + h1(visited_node.state):
            child.parent = node
            visited_node.parent = None

  return None, count

In [ ]:
if IS_VALID:
  puzzle = n_puzzle(board['start'], board['goal'])

  print('Start:')
  print(np.matrix(board['start']))
  print('Goal:')
  print(np.matrix(board['goal']))
  path, count = h1_a_star_search(puzzle)

  if path is not None:
    print(f"Solution length: {len(path)}")
    print(f"Path: {path}")
    print(f"States examined: {count}")
else:
  print('Invalid puzzle data')

Start:
[[7 2 4]
 [5 0 6]
 [8 3 1]]
Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
Solution length: 26
Path: ['Left', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left']
States examined: 32644


### A* using h2 (Manhattan Distance)

In [ ]:
def h2(node_state, goal_state):

  h = 0
  node_state = np.array(node_state)
  goal_state = np.array(goal_state)

  for i in range(1, 8):
    board = np.where(node_state==i)
    node_y, node_x = board[0][0], board[1][0]

    goal_board = np.where(goal_state ==i)
    goal_y, goal_x = goal_board[0][0], goal_board[1][0]

    h += (abs(node_y - goal_y) + abs(node_x - goal_x))

  return h

In [ ]:
def h2_a_star_search(problem):

  # Initialize open with starting state
  open = [(0, problem.initial, Node(problem.initial))]
  closed = set()
  count = 0

  # Loop as long as there are nodes in frontier
  while open:

    count += 1

    # Pop first element
    cost, current_state, node = hq.heappop(open)
    closed.add(tuple(map(tuple, current_state)))

    # If goal state found, return path and number of states examined
    if is_goal_state(current_state, problem.goal):
      return path_back(node), count

    # Get possible moves
    rules = actions(current_state)

    # Create lists of board states
    open_states = [state for _, state, obj in open]

    for rule in rules: # Expand frontier

      new_state = successor_state(current_state, rule)

      child = Node(new_state, node, rule)

      # If the resultant state is not in open or closed, add to open
      if new_state not in open_states and tuple(map(tuple,new_state)) not in closed:
        heuristic_val = child.cost + h2(child.state, problem.goal)
        hq.heappush(open, (heuristic_val, child.state, child))

      # If the new state is in the closed set,
      # check if shorter path has been found
      if tuple(map(tuple,new_state)) in closed:
        visited_node = search_path(child, new_state)

        if visited_node is not None:
          heuristic_val = child.cost + h2(new_state, problem.goal)

          if heuristic_val < visited_node.cost + h2(visited_node.state, problem.goal):
            child.parent = node
            visited_node.parent = None

  return None, count

In [ ]:
if IS_VALID:
  puzzle = n_puzzle(board['start'], board['goal'])

  print('Start:')
  print(np.matrix(board['start']))
  print('Goal:')
  print(np.matrix(board['goal']))
  path, count = h2_a_star_search(puzzle)

  if path is not None:
    print(f"Solution length: {len(path)}")
    print(f"Path: {path}")
    print(f"States examined: {count}")
else:
  print('Invalid puzzle data')

Start:
[[7 2 4]
 [5 0 6]
 [8 3 1]]
Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]
Solution length: 26
Path: ['Left', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left']
States examined: 4634


# Report

**Solution found for 2-moves problem using graph search**:

Start:
[[3 1 2]
 [4 0 5]
 [6 7 8]]

Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]

Solution length: 2

Path: ['Left', 'Up']

States examined: 6

**Solution found for 5-moves problem using graph search**:

Start:
[[3 1 2]
 [4 7 0]
 [6 8 5]]

Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]

Solution length: 5

Path: ['Down', 'Left', 'Up', 'Left', 'Up']

States examined: 34

**Solution found for 10-moves problem using graph search**:

Start:
[[3 7 1]
 [6 4 2]
 [0 8 5]]

Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]

Solution length: 10

Path: ['Up', 'Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Up']

States examined: 421

**Solution found for 15-moves problem using graph search**:

Start:
[[3 7 1]
 [6 2 5]
 [8 0 4]]

Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]

Solution length: 15

Path: ['Right', 'Up', 'Left', 'Down', 'Left', 'Up', 'Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Up']

States examined: 5174

**Solution found for 20-moves problem using graph search**:

Start:
[[7 1 0]
 [3 6 5]
 [8 2 4]]

Goal:
[[0 1 2]
 [3 4 5]
 [6 7 8]]

Solution length: 20

Path: ['Down', 'Left', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Up', 'Left', 'Down', 'Right', 'Up', 'Right', 'Up', 'Left', 'Left']

States examined: 37810

**Solution for 25-moves problem using graph search**



## Sources

- https://notebook.community/Chipe1/aima-python/search
- https://github.com/aimacode/aima-python/blob/master/search.py#L426